[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abderrahmane-mhd/text-anonymization/blob/main/TextAnonymization.ipynb)


# Text Anonymization


The upcoming notebook will showcase the implementation of an anonymization pipeline, utilizing Presidio - a comprehensive anonymization library. During the development of the pipeline, we will compare and evaluate various methods of text anonymization, in order to determine the most effective approach for the given task.

* The first method we will explore is the use of Named Entity Recognition (NER) models, implemented using both Spacy and Transformers frameworks. NER models are capable of identifying and labeling specific types of named entities in text, such as names, locations, and organizations, which can then be replaced with anonymized placeholders.

* Next, we will also explore the use of a Part-Of-Speech (POS) tagging model, implemented using Transformers, which labels each word in a given text with a corresponding part of speech. This can help identify certain words that should be anonymized, such as verbs, adjectives, or adverbs.

* Finally, we will integrate both Transformers and Spacy models to create a hybrid approach to text anonymization, which may yield better results than using either method independently. By comparing the performance and effectiveness of each approach, we aim to provide valuable insights into the most effective techniques for anonymizing sensitive data.

PS: In the following notebook, I've assessed the anonymization model's performance on two distinct text categories: narrative-style text and call recording transcripts, as these represent typical applications for NLP models.

We have started by generating a French Text with annotations using the following prompt in ChatGPT app

In [1]:
text_fr = """Il était une fois, dans la ville de Paris, une jeune femme nommée Marie qui travaillait pour l'Organisation des Nations unies pour l'alimentation et l'agriculture (FAO). Elle avait pour mission de se rendre dans différents pays pour étudier les pratiques agricoles locales et promouvoir des méthodes durables pour nourrir la population mondiale croissante. Marie était passionnée par son travail et avait déjà visité de nombreux endroits, tels que le Brésil, le Kenya et l'Inde.

Un jour, alors qu'elle se trouvait en mission en Égypte, Marie rencontra un homme charmant du nom d'Ahmed. Il travaillait pour une organisation locale appelée l'Association égyptienne pour le développement rural (AEDR), qui avait pour objectif d'améliorer les conditions de vie des communautés rurales en Égypte. Ahmed était également très intéressé par l'agriculture durable et lui et Marie se lièrent d'amitié rapidement.

Marie et Ahmed ont continué à travailler ensemble pour promouvoir des pratiques agricoles durables en Égypte et ont finalement créé une organisation conjointe appelée l'Initiative pour une agriculture durable (IAS). Ils ont réussi à obtenir un financement de l'Union européenne pour leur projet et ont pu étendre leur travail à d'autres pays du Moyen-Orient.

Au fil des années, l'IAS est devenue une organisation de premier plan dans le domaine de l'agriculture durable et a reçu de nombreux prix pour son travail. Marie et Ahmed ont continué à diriger l'organisation ensemble et ont fondé une famille heureuse en Égypte. Leur travail a permis d'améliorer la vie de nombreuses personnes dans le monde entier et leur héritage continue de vivre à travers l'IAS, qui continue à promouvoir des pratiques agricoles durables et à lutter contre la faim dans le monde."""

In [2]:
annotations = {"Paris": "LOC", "Marie": "PER", "Organisation des Nations unies pour l'alimentation et l'agriculture (FAO)": "ORG", "Brésil": "LOC", "Kenya": "LOC", "Inde": "LOC", "Égypte": "LOC", "Ahmed": "PER", "Association égyptienne pour le développement rural (AEDR)": "ORG", "Initiative pour une agriculture durable (IAS)": "ORG", "Union européenne": "ORG", "Moyen-Orient": "LOC"}


In [3]:
new_text_fr = """Bonjour, Jean Dupont ici. Je suis intéressé par l'achat d'un nouvel appareil électronique. Je me demandais si vous pourriez m'aider à trouver le meilleur modèle pour mes besoins.

Bonjour Monsieur Dupont, je suis Sophie Martin de la société Techno Plus. Bien sûr, je serais heureuse de vous aider. Quels sont les spécifications techniques que vous recherchez ?

Eh bien, je cherche un modèle avec une grande capacité de stockage, une haute résolution d'écran et un processeur rapide.

D'accord, nous avons plusieurs modèles qui pourraient correspondre à ces spécifications. L'un d'eux est notre modèle haut de gamme, qui dispose d'un écran OLED et d'un processeur quad-core. Cependant, il est plus cher que nos modèles standard.

Je vois. Et quels sont les autres modèles disponibles ?

Nous avons également notre modèle standard, qui a une capacité de stockage de base mais une résolution d'écran similaire. Il est moins cher que notre modèle haut de gamme. Nous avons également un modèle intermédiaire qui est un peu plus cher que le modèle standard mais offre une meilleure capacité de stockage.

Hmm, c'est difficile de choisir. Pourriez-vous me donner le prix de chacun de ces modèles ?

Bien sûr, le modèle standard est à 599 euros, le modèle intermédiaire est à 799 euros et le modèle haut de gamme est à 999 euros.

Très bien, merci pour ces informations. Et pouvez-vous me dire où est située votre entreprise ?

Nous sommes basés à Tokyo, mais nous avons des centres de distribution dans le monde entier.

D'accord, merci. Et pour la commande, comment ça se passe ?

Vous pouvez commander en ligne sur notre site Web, ou vous pouvez nous appeler directement pour passer une commande. Si vous voulez commander par téléphone, voici notre numéro : +33 1 23 45 67 89.

Très bien, je prends note de ça. Et pour la livraison, je peux la faire livrer à une adresse spécifique ?

Oui, bien sûr. Vous pouvez nous donner l'adresse de livraison et nous nous occuperons du reste.

Parfait, merci beaucoup pour votre aide, Sophie.

De rien, c'était un plaisir de vous aider, Monsieur Dupont. Si vous avez d'autres questions, n'hésitez pas à nous contacter à nouveau."""

## Library Installation


In [4]:
!pip install presidio-analyzer
!pip install presidio-anonymizer
!python -m spacy download fr_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s eta 0:00:00
2023-09-02 17:45:53.445397: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 17:45:54.744389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00


In [6]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00


## 1. NER SpaCy Model

We will start by trying NER model proposed by SpaCy

In [7]:
import spacy

nlp = spacy.load("fr_core_news_lg")

In [8]:
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer import AnalyzerEngine, EntityRecognizer, RecognizerResult, Pattern, PatternRecognizer

from presidio_analyzer.nlp_engine import NlpArtifacts,NlpEngineProvider


In [9]:
configuration = {"nlp_engine_name":"spacy", "models":[{"lang_code":"fr", "model_name":"fr_core_news_lg"}]}


provider = NlpEngineProvider(nlp_configuration=configuration)

nlp_engine = provider.create_engine()


analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine,
    supported_languages = ['fr']
)

In [10]:
result = analyzer.analyze(text=text_fr, language='fr')

print(result)

[type: LOCATION, start: 36, end: 41, score: 0.85, type: PERSON, start: 66, end: 71, score: 0.85, type: PERSON, start: 357, end: 362, score: 0.85, type: LOCATION, start: 451, end: 457, score: 0.85, type: LOCATION, start: 462, end: 467, score: 0.85, type: LOCATION, start: 473, end: 477, score: 0.85, type: LOCATION, start: 529, end: 535, score: 0.85, type: PERSON, start: 537, end: 542, score: 0.85, type: PERSON, start: 580, end: 585, score: 0.85, type: LOCATION, start: 785, end: 791, score: 0.85, type: PERSON, start: 793, end: 798, score: 0.85, type: PERSON, start: 866, end: 871, score: 0.85, type: PERSON, start: 905, end: 910, score: 0.85, type: PERSON, start: 914, end: 919, score: 0.85, type: LOCATION, start: 1007, end: 1013, score: 0.85, type: LOCATION, start: 1250, end: 1262, score: 0.85, type: PERSON, start: 1421, end: 1426, score: 0.85, type: PERSON, start: 1430, end: 1435, score: 0.85, type: LOCATION, start: 1520, end: 1526, score: 0.85]


In [11]:
found_entities = [text_fr[obj.to_dict()['start']:obj.to_dict()['end']] for obj in result]

unique_entities = set(found_entities)

print(unique_entities)

{'Inde', 'Marie', 'Paris', 'Égypte', 'Kenya', 'Ahmed', 'Moyen-Orient', 'Brésil'}


In [12]:
set(annotations)

{'Ahmed',
 'Association égyptienne pour le développement rural (AEDR)',
 'Brésil',
 'Inde',
 'Initiative pour une agriculture durable (IAS)',
 'Kenya',
 'Marie',
 'Moyen-Orient',
 "Organisation des Nations unies pour l'alimentation et l'agriculture (FAO)",
 'Paris',
 'Union européenne',
 'Égypte'}

In [13]:
accuracy = len(set(unique_entities) & set(annotations)) / len(annotations)
print(f"Accuracy score: {accuracy:.2f}")

Accuracy score: 0.67


## NER Transformers Model

In this section we try out a Name Entity Recognition Model using a transformer model CamemBERT adapted for French Text

In [16]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
# Loading both tokenizer and NER model
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
ner_model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

In [17]:
nlp = pipeline('ner', model=ner_model, tokenizer=tokenizer, aggregation_strategy="simple")

transformer_res = nlp(text_fr)

In [18]:
detected_entities = [res['word'] for res in transformer_res]

In [19]:
set(detected_entities)

{'AEDR',
 'Ahmed',
 'Association égyptienne pour le développement rural',
 'Brésil',
 'FAO',
 'IAS',
 'Inde',
 'Initiative pour une agriculture durable',
 'Kenya',
 'Marie',
 'Moyen-Orient',
 "Organisation des Nations unies pour l'alimentation et l'agriculture",
 'Paris',
 'Union européenne',
 'Égypte'}

Although the NER transformer model was successful in detecting all the entities, it identified some organizations and abbreviations separately, such as "*Initiative pour une agriculture durable*" and "*IAS*". Despite this difference, it doesn't impact the overall anonymization process since the identified entities will still be anonymized.

## Mixed Pipeline development (Transformers + SpaCy)

In [20]:
from transformers import pipeline
# list of entities: https://microsoft.github.io/presidio/supported_entities/#list-of-supported-entities
DEFAULT_ANOYNM_ENTITIES = [
    "CREDIT_CARD",
    "CRYPTO",
    "DATE_TIME",
    "EMAIL_ADDRESS",
    "IBAN_CODE",
    "IP_ADDRESS",
    "NRP",
    "LOCATION",
    "PERSON",
    "PHONE_NUMBER",
    "MEDICAL_LICENSE",
    "URL",
    "ORGANIZATION",
    "NUMBER"
]

class TransformerRecognizer(EntityRecognizer):
    def __init__(
        self,
        model_id_or_path,
        mapping_labels,
        aggregation_strategy="simple",
        supported_language="fr",
        ignore_labels=["O", "MISC"],
    ):
        # inits transformers pipeline for given mode or path
        self.pipeline = pipeline(
            "token-classification", model=model_id_or_path, aggregation_strategy=aggregation_strategy, ignore_labels=ignore_labels
        )
        # map labels to presidio labels
        self.label2presidio = mapping_labels

        # passes entities from model into parent class
        super().__init__(supported_entities=list(self.label2presidio.values()), supported_language=supported_language)

    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(
        self, text: str, entities = None, nlp_artifacts: NlpArtifacts = None
    ):
        """
        Extracts entities using Transformers pipeline
        """
        results = []

        predicted_entities = self.pipeline(text)
        if len(predicted_entities) > 0:
            for e in predicted_entities:
                if(e['entity_group'] not in self.label2presidio):
                    continue
                converted_entity = self.label2presidio[e["entity_group"]]
                if converted_entity in entities or entities is None:
                    results.append(
                        RecognizerResult(
                            entity_type=converted_entity, start=e["start"], end=e["end"], score=e["score"]
                        )
                    )
        return results

In [21]:
#mapping_labels = {"PROPN": "PERSON","XFAMIL": "PERSON"}
mapping_labels = {"PER":"PERSON",'LOC':'LOCATION','ORG':"ORGANIZATION",'PHONE_NUMBER':'PHONE_NUMBER'}
configuration = {"nlp_engine_name":"spacy",
                "models":[{"lang_code": 'fr', "model_name":"fr_core_news_lg"}]}


to_keep = []
lang = 'fr'

In [22]:
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine,
    supported_languages = "fr"
)

transformers_recognizer = TransformerRecognizer("Jean-Baptiste/camembert-ner", mapping_labels)
analyzer.registry.add_recognizer(transformers_recognizer)

In [23]:
# Text Analyzer
analyzer_results = analyzer.analyze(text=new_text_fr, entities = DEFAULT_ANOYNM_ENTITIES, allow_list = to_keep, language=lang)

# Text Anonymizer
engine = AnonymizerEngine()
result = engine.anonymize(text=new_text_fr, analyzer_results=analyzer_results)

# Restructuring anonymizer results

anonymization_results =  {"anonymized": result.text,"found": [entity.to_dict() for entity in analyzer_results]}

words = [{'word': new_text_fr[obj['start']:obj['end']], 'entity_type':obj['entity_type'], 'start':obj['start'], 'end':obj['end']} for obj in anonymization_results['found']]

In [24]:
words

[{'word': ' Tokyo', 'entity_type': 'LOCATION', 'start': 1440, 'end': 1446},
 {'word': ' Sophie Martin', 'entity_type': 'PERSON', 'start': 212, 'end': 226},
 {'word': ' Techno Plus',
  'entity_type': 'ORGANIZATION',
  'start': 240,
  'end': 252},
 {'word': ' Monsieur Dupont',
  'entity_type': 'PERSON',
  'start': 187,
  'end': 203},
 {'word': ' Monsieur Dupont',
  'entity_type': 'PERSON',
  'start': 2070,
  'end': 2086},
 {'word': ' Jean Dupont', 'entity_type': 'PERSON', 'start': 8, 'end': 20},
 {'word': ' Sophie', 'entity_type': 'PERSON', 'start': 2018, 'end': 2025}]

In [25]:
# We had to strip the results to remove the leading spaces and \n
word_results = [res['word'].strip() for res in words]

In [26]:
set(word_results)

{'Jean Dupont',
 'Monsieur Dupont',
 'Sophie',
 'Sophie Martin',
 'Techno Plus',
 'Tokyo'}

In [27]:
anonymization_results['anonymized']

"Bonjour,<PERSON> ici. Je suis intéressé par l'achat d'un nouvel appareil électronique. Je me demandais si vous pourriez m'aider à trouver le meilleur modèle pour mes besoins.\n\nBonjour<PERSON>, je suis<PERSON> de la société<ORGANIZATION>. Bien sûr, je serais heureuse de vous aider. Quels sont les spécifications techniques que vous recherchez ?\n\nEh bien, je cherche un modèle avec une grande capacité de stockage, une haute résolution d'écran et un processeur rapide.\n\nD'accord, nous avons plusieurs modèles qui pourraient correspondre à ces spécifications. L'un d'eux est notre modèle haut de gamme, qui dispose d'un écran OLED et d'un processeur quad-core. Cependant, il est plus cher que nos modèles standard.\n\nJe vois. Et quels sont les autres modèles disponibles ?\n\nNous avons également notre modèle standard, qui a une capacité de stockage de base mais une résolution d'écran similaire. Il est moins cher que notre modèle haut de gamme. Nous avons également un modèle intermédiaire q

There may be situations where Named Entity Recognition (NER) models are unable to identify certain words that need to be anonymized. In such cases, Part-Of-Speech (POS) Tagging can be used as an alternative approach to ensure stricter anonymization. POS tagging involves labeling each word in a text with a corresponding part of speech, such as noun, verb, adjective, or adverb. This can help in identifying specific types of words that need to be anonymized, such as names, locations, or organizations. By combining the results of both NER and POS tagging, we can achieve a more comprehensive and accurate approach to anonymization, which is particularly important in cases where data privacy is a concern.